In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
sdir = r'G:\Dataset-image-Eye_Data_New'

slist = os.listdir(sdir)
classes = []
filepaths = []
labels = []

for d in slist:
    dpath = os.path.join(sdir, d)
    if d != "Test" and d != ".git":
        if os.path.isdir(dpath):
            classes.append(d)

class_count = len(classes)

for klass in classes:
    classpath = os.path.join(sdir, klass)
    filelist = os.listdir(classpath)
    for f in filelist:
        fpath = os.path.join(classpath, f)
        filepaths.append(fpath)
        labels.append(klass)

print('number of files: ', len(filepaths), '   number of labels: ', len(labels))

file_series = pd.Series(filepaths, name='filepaths')
label_series = pd.Series(labels, name='labels')
df = pd.concat([file_series, label_series], axis=1)
print(df.head())


number of files:  16016    number of labels:  16016
                                           filepaths       labels
0  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
1  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
2  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
3  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus
4  G:\Dataset-image-Eye_Data_New\Keratoconus\KCN_...  Keratoconus


In [3]:
balance = df['labels'].value_counts()
print(balance)

train_split = .8
test_split = .1
dummy_split = test_split / (1 - train_split)

train_df, dummy_df = train_test_split(df, train_size=train_split, shuffle=True, random_state=125)
test_df, valid_df = train_test_split(dummy_df, train_size=dummy_split, shuffle=True, random_state=125)

print('train size: ', len(train_df), '  test size: ', len(test_df), '   valid size: ', len(valid_df))
length = len(test_df)


labels
Normal         5600
Keratoconus    5572
Suspect        4844
Name: count, dtype: int64
train size:  12812   test size:  1602    valid size:  1602


In [4]:
batch_size = 16

def scalar(x):
    return x / 127.5 - 1

trgen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=scalar,
    horizontal_flip=True
)

train_gen = trgen.flow_from_dataframe(
    train_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(448, 448),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=123
)

tvgen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=scalar
)

valid_gen = tvgen.flow_from_dataframe(
    valid_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(448, 448),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

test_batch_size = sorted(
    [int(length / n) for n in range(1, length + 1)
     if length % n == 0 and length / n <= batch_size],
    reverse=True
)[0]

test_steps = int(length / test_batch_size)

test_gen = tvgen.flow_from_dataframe(
    test_df,
    x_col='filepaths',
    y_col='labels',
    target_size=(448, 448),
    class_mode='categorical',
    batch_size=test_batch_size,
    shuffle=False
)

test_labels = test_gen.labels


Found 12812 validated image filenames belonging to 3 classes.
Found 1602 validated image filenames belonging to 3 classes.
Found 1602 validated image filenames belonging to 3 classes.


In [5]:
def show_training_samples(gen):
    class_dict = gen.class_indices
    new_dict = {}
    for key, value in class_dict.items():
        new_dict[value] = key

    images, labels = next(gen)
    plt.figure(figsize=(15, 15))
    length = len(labels)
    if length < 25:
        r = length
    else:
        r = 25

    for i in range(r):
        plt.subplot(5, 5, i + 1)
        image = (images[i] + 1) / 2
        plt.imshow(image)
        index = np.argmax(labels[i])
        class_name = new_dict[index]
        plt.title(class_name, color='blue', fontsize=16)
        plt.axis('off')
    plt.show()


In [6]:
img_shape = (448, 448, 3)
neurons = 512
dropout = .3
lr = .001
freeze = True

base_model = tf.keras.applications.VGG19(include_top=False, input_shape=img_shape, pooling='max', weights='imagenet')

if freeze:
    base_model.trainable = False

x = base_model.output
x = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)

x = tf.keras.layers.Dense(
    neurons,
    kernel_regularizer=regularizers.l2(0.016),
    activity_regularizer=regularizers.l1(0.006),
    bias_regularizer=regularizers.l1(0.006),
    activation='relu',
    kernel_initializer=tf.keras.initializers.GlorotUniform(seed=123)
)(x)

x = tf.keras.layers.Dropout(rate=dropout, seed=123)(x)

output = tf.keras.layers.Dense(
    class_count,
    activation='softmax',
    kernel_initializer=tf.keras.initializers.GlorotUniform(seed=123)
)(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
def print_in_color(txt_msg, fore_tupple, back_tupple):
    rf, gf, bf = fore_tupple
    rb, gb, bb = back_tupple
    msg = '{0}' + txt_msg
    mat = '\33[38;2;' + str(rf) + ';' + str(gf) + ';' + str(bf) + ';48;2;' + str(rb) + ';' + str(gb) + ';' + str(bb) + 'm'
    print(msg.format(mat), flush=True)
    print('\33[0m', flush=True)
    return


In [8]:
class LRA(keras.callbacks.Callback):
    def __init__(self, patience, stop_patience, threshold, factor, dwell, model_name, freeze, end_epoch):
        super(LRA, self).__init__()
        self.patience = patience
        self.stop_patience = stop_patience
        self.threshold = threshold
        self.factor = factor
        self.dwell = dwell
        self.lr = 0
        self.highest_tracc = 0.0
        self.lowest_vloss = np.inf
        self.count = 0
        self.stop_count = 0
        self.end_epoch = end_epoch
        self.best_weights = None

        msg = ' '
        if freeze:
            msgs = f' Starting training using base model {model_name} with weights frozen...'
        else:
            msgs = f' Starting training using base model {model_name} training all layers '
        print_in_color(msgs, (244, 252, 3), (55, 65, 80))

    def set_model(self, model):
        self.model_ = model
        self.lr = float(tf.keras.backend.get_value(model.optimizer.learning_rate))
        self.best_weights = self.model_.get_weights()

    def on_epoch_begin(self, epoch, logs=None):
        if epoch != 0:
            msgs = f'for epoch {epoch} '
            msgs = msgs + LRA.msg
            print_in_color(msgs, (255, 255, 0), (55, 65, 80))

    def on_epoch_end(self, epoch, logs=None):
        lr = float(tf.keras.backend.get_value(self.model_.optimizer.learning_rate))
        v_loss = logs.get('val_loss')
        acc = logs.get('accuracy')

        if acc < self.threshold:
            if acc > self.highest_tracc:
                LRA.msg = f' training accuracy improved from {self.highest_tracc} to {acc}...'
                self.highest_tracc = acc
                LRA.best_weights = self.model_.get_weights()
                self.count = 0
                self.stop_count = 0
                if v_loss < self.lowest_vloss:
                    self.lowest_vloss = v_loss
            else:
                if self.count >= self.patience - 1:
                    self.lr = lr * self.factor
                    self.model_.optimizer.learning_rate.assign(self.lr)
                    self.count = 0
                    self.stop_count = self.stop_count + 1
                    if self.dwell:
                        self.model_.set_weights(LRA.best_weights)
                else:
                    self.count = self.count + 1
        else:
            if v_loss < self.lowest_vloss:
                self.lowest_vloss = v_loss
                LRA.best_weights = self.model_.get_weights()
                self.count = 0
                self.stop_count = 0
            else:
                if self.count >= self.patience - 1:
                    self.lr = self.lr * self.factor
                    self.stop_count = self.stop_count + 1
                    self.count = 0
                    self.model_.optimizer.learning_rate.assign(self.lr)
                    if self.dwell:
                        self.model_.set_weights(LRA.best_weights)
                else:
                    self.count = self.count + 1

        if epoch == self.end_epoch:
            print_in_color(LRA.msg, (255, 255, 0), (55, 65, 80))

        if self.stop_count > self.stop_patience - 1:
            LRA.msg = f' training halted at epoch {epoch + 1}...'
            print_in_color(LRA.msg, (0, 255, 0), (55, 65, 80))
            self.model_.stop_training = True


In [9]:
patience=1
stop_patience=4
threshold=.9
factor=.5
dwell=False
model_type='ResNet50'
epochs=5
callbacks=[LRA(patience=patience,stop_patience=stop_patience, threshold=threshold, factor=factor,dwell=dwell, model_name=model_type, freeze=freeze, end_epoch=epochs - 1 )]

history=model.fit(x=train_gen,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=valid_gen, validation_steps=None,  shuffle=False,  initial_epoch=0)

 Starting training using base model ResNet50 with weights frozen...

Epoch 1/5
801/801 ━━━━━━━━━━━━━━━━━━━━ 5270s 7s/step - accuracy: 0.5165 - loss: 6.7364 - val_accuracy: 0.5624 - val_loss: 2.2777
for epoch 1  training accuracy improved from 0.0 to 0.5164689421653748...

Epoch 2/5
801/801 ━━━━━━━━━━━━━━━━━━━━ 5367s 7s/step - accuracy: 0.5276 - loss: 1.4419 - val_accuracy: 0.6030 - val_loss: 1.0458
for epoch 2  training accuracy improved from 0.5164689421653748 to 0.5276303291320801...

Epoch 3/5
801/801 ━━━━━━━━━━━━━━━━━━━━ 5291s 7s/step - accuracy: 0.5596 - loss: 1.0113 - val_accuracy: 0.6180 - val_loss: 0.9519
for epoch 3  training accuracy improved from 0.5276303291320801 to 0.5595535635948181...

Epoch 4/5
801/801 ━━━━━━━━━━━━━━━━━━━━ 5398s 7s/step - accuracy: 0.5717 - loss: 0.9656 - val_accuracy: 0.6330 - val_loss: 0.9125
for epoch 4  training accuracy improved from 0.5595535635948181 to 0.5716515779495239...

Epoch 5/5
801/801 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5814 -